# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [3]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
df.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [6]:
# more frindly view
for ship in df.head(5):
    print (ship)
    print ('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)




In [8]:
# how many ships by cruise line
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = indexer.fit(df).transform(df)
indexed.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [11]:
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [17]:
indexed.select(['Cruise_line','cruise_cat']).show()

+-----------+----------+
|Cruise_line|cruise_cat|
+-----------+----------+
|    Azamara|      16.0|
|    Azamara|      16.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
|   Carnival|       1.0|
+-----------+----------+
only showing top 20 rows



In [18]:
# moving on working with VectorAssembler and get the Liner Regression working
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [19]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [25]:
# creating an unique column 'features' from data input columns 
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'], outputCol='features')

In [26]:
output = assembler.transform(indexed)

In [27]:
# the data shows direct relation between collected features and crew
output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [29]:
final_data = output.select(['features','crew'])

In [58]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [59]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               107|
|   mean| 7.634485981308423|
| stddev|3.4170174111363827|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [60]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               51|
|   mean|8.129215686274511|
| stddev|3.690456255336056|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [61]:
# build regression model
from pyspark.ml.regression import LinearRegression

In [62]:
ship_lr = LinearRegression(labelCol='crew')

In [63]:
trained_ship_model = ship_lr.fit(train_data)

In [64]:
ship_results = trained_ship_model.evaluate(test_data)

In [65]:
# this result can be affected by randomSplit parameters 0.7 and 0.3 from training and testing
ship_results.rootMeanSquaredError

0.7975096169475563

In [66]:
# this result can be affected by randomSplit parameters 0.7 and 0.3 from training and testing
ship_results.r2

0.9523665644800545

In [67]:
ship_results.meanSquaredError

0.636021589123838

In [69]:
ship_results.meanAbsoluteError

0.6007456408527734

In [70]:
from pyspark.sql.functions import corr

In [71]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [73]:
# this correlation makes sense, the more passengers the more crew
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [74]:
# even better correlation between number of cabins and crew.
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [75]:
# let's predict some crew from test data
unlabeled_data = test_data.select('features')

In [76]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,86.0,21.04,9...|
|[5.0,115.0,35.74,...|
|[5.0,122.0,28.5,1...|
|[6.0,30.276999999...|
|[6.0,90.0,20.0,9....|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[9.0,110.0,29.74,...|
|[9.0,113.0,26.74,...|
|[9.0,116.0,26.0,9...|
|[10.0,58.825,15.6...|
|[10.0,77.0,20.16,...|
|[10.0,81.76899999...|
|[10.0,91.62700000...|
|[10.0,105.0,27.2,...|
|[10.0,110.0,29.74...|
|[11.0,90.0,22.4,9...|
|[11.0,90.09,25.01...|
|[12.0,58.6,15.66,...|
+--------------------+
only showing top 20 rows



In [77]:
predictions = trained_ship_model.transform(unlabeled_data)

In [78]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.159935441757884|
|[5.0,86.0,21.04,9...|  9.27919047351494|
|[5.0,115.0,35.74,...|12.052063813534115|
|[5.0,122.0,28.5,1...| 5.387293658900061|
|[6.0,30.276999999...| 4.448991704728874|
|[6.0,90.0,20.0,9....| 10.29844290728652|
|[8.0,110.0,29.74,...|12.288164976573082|
|[9.0,88.5,21.24,9...|  9.56566380821488|
|[9.0,110.0,29.74,...|12.275627150876177|
|[9.0,113.0,26.74,...|11.385481080284814|
|[9.0,116.0,26.0,9...|11.160816523751453|
|[10.0,58.825,15.6...| 7.332459491216202|
|[10.0,77.0,20.16,...| 8.893339244716756|
|[10.0,81.76899999...| 8.758651194561256|
|[10.0,91.62700000...| 9.150978466958598|
|[10.0,105.0,27.2,...| 11.38985750925571|
|[10.0,110.0,29.74...|12.255524575904968|
|[11.0,90.0,22.4,9...| 10.07787869027373|
|[11.0,90.09,25.01...| 8.729286330345401|
|[12.0,58.6,15.66,...| 7.466274618373834|
+--------------------+------------